<a href="https://colab.research.google.com/github/wfreinhart/sdmm-regression/blob/main/notebooks/hyperopt_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=31e7aa02be7e4f4e6433239a47fa2e979ab9e907f3e7c93914219e8f5b12f6ce
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

## raw data

In [ ]:
import numpy as np
import pandas as pd
import json

with open('data-10-folds.json', 'r') as fid:
    buffer = json.load(fid)

X = np.array(buffer['X'])
y = np.array(buffer['y'])

fold_data = []
for fold in buffer['folds']:
    train_index = np.array(fold['train'])
    test_index = np.array(fold['test'])

    trainX, testX = X[train_index], X[test_index]
    trainy, testy = y[train_index], y[test_index]
    
    fold_data.append({'train': {'X': trainX, 'y': trainy},
                      'test': {'X': testX, 'y': testy}
                      })

## k-mer tokens

In [ ]:
import itertools
import tqdm.notebook
from math import factorial


def swap_monomers(seq):
    seq = seq.replace('A', 'C')
    seq = seq.replace('B', 'A')
    seq = seq.replace('C', 'B')
    return seq


def make_base(degree):
    # construct unique numbers of A/B monomers
    base = []
    for i in range(degree+1):
        for j in range(i):
            seq = ''.join(['A' for _ in range(i-j)] + ['B' for _ in range(j)])
            # print((i, j), seq)
            base += [''.join(x) for x in itertools.permutations(seq)]
            base += [''.join(x) for x in itertools.permutations(swap_monomers(seq))]
    base = sorted(set(base))
    pruned_base = []
    for b in base:
        if b not in pruned_base and b[::-1] not in pruned_base:
            pruned_base.append(b)
    base = pruned_base
    print(f'Finding {len(base)} patterns:', base)

    return base


def featurize(chain_sequences, base):
    X = np.zeros([len(chain_sequences), len(base)])
    for i, chain in tqdm.notebook.tqdm(enumerate(chain_sequences), total=len(chain_sequences)):
        if 'A' in str(chain):
            seq = chain
        else:
            seq = ''.join(['A' if x == 0 else 'B' for x in chain])
        X[i] = 0.5 * (np.array([seq.count(b) for b in base]) + np.array([seq[::-1].count(b) for b in base]))
    return X

## create training fold data with k-mers

In [ ]:
base = make_base(degree=10)
token_X = featurize(X, base)

token_fold_data = []
for fold in buffer['folds']:
    train_index = fold['train']
    test_index = fold['test']

    trainX, testX = token_X[train_index], token_X[test_index]
    trainy, testy = y[train_index], y[test_index]
    
    token_fold_data.append({'train': {'X': trainX, 'y': trainy},
                            'test': {'X': testX, 'y': testy}
                           })

Finding 1085 patterns: ['A', 'AA', 'AAA', 'AAAA', 'AAAAA', 'AAAAAA', 'AAAAAAA', 'AAAAAAAA', 'AAAAAAAAA', 'AAAAAAAAAA', 'AAAAAAAAAB', 'AAAAAAAAB', 'AAAAAAAABA', 'AAAAAAAABB', 'AAAAAAAB', 'AAAAAAABA', 'AAAAAAABAA', 'AAAAAAABAB', 'AAAAAAABB', 'AAAAAAABBA', 'AAAAAAABBB', 'AAAAAAB', 'AAAAAABA', 'AAAAAABAA', 'AAAAAABAAA', 'AAAAAABAAB', 'AAAAAABAB', 'AAAAAABABA', 'AAAAAABABB', 'AAAAAABB', 'AAAAAABBA', 'AAAAAABBAA', 'AAAAAABBAB', 'AAAAAABBB', 'AAAAAABBBA', 'AAAAAABBBB', 'AAAAAB', 'AAAAABA', 'AAAAABAA', 'AAAAABAAA', 'AAAAABAAAA', 'AAAAABAAAB', 'AAAAABAAB', 'AAAAABAABA', 'AAAAABAABB', 'AAAAABAB', 'AAAAABABA', 'AAAAABABAA', 'AAAAABABAB', 'AAAAABABB', 'AAAAABABBA', 'AAAAABABBB', 'AAAAABB', 'AAAAABBA', 'AAAAABBAA', 'AAAAABBAAA', 'AAAAABBAAB', 'AAAAABBAB', 'AAAAABBABA', 'AAAAABBABB', 'AAAAABBB', 'AAAAABBBA', 'AAAAABBBAA', 'AAAAABBBAB', 'AAAAABBBB', 'AAAAABBBBA', 'AAAAABBBBB', 'AAAAB', 'AAAABA', 'AAAABAA', 'AAAABAAA', 'AAAABAAAA', 'AAAABAAAAB', 'AAAABAAAB', 'AAAABAAABA', 'AAAABAAABB', 'AAAABAAB', 'AA

  0%|          | 0/2038 [00:00<?, ?it/s]

# scikit-learn models

In [ ]:
from sklearn import linear_model
from sklearn import neighbors
from sklearn import ensemble

models = [linear_model.LinearRegression(),
          linear_model.Lasso(),
          linear_model.Ridge(),
          neighbors.KNeighborsRegressor(),
          ensemble.RandomForestRegressor(random_state=0)]

for model in models:
    print(model)
    metrics = train_evaluate_model(model, dataset)
    print(metrics)

# for model in models:
#     plot_parity(model.predict, dataset, name=str(model))

## optimize linear models

In [ ]:
from sklearn import linear_model


this_data = token_fold_data[0]

def loss_fn(**params):
    # params = {'alpha': np.exp(params['log_alpha'])}

    model = linear_model.Ridge(**params)

    this_X = this_data['train']['X']
    this_y = this_data['train']['y']

    train_X = np.vstack([this_X, np.fliplr(this_X)])
    train_y = np.vstack([this_y, this_y])

    model.fit(train_X, train_y)

    test_X = this_data['test']['X']
    test_y = this_data['test']['y']

    # forward sequences
    pred_z_fwd = model.predict(test_X)

    # reverse sequences
    pred_z_rev = model.predict(np.fliplr(test_X))

    # averarage the two predictions
    rmse_avg = np.sqrt(np.mean((0.5*(pred_z_fwd + pred_z_rev) - test_y)**2))

    return -rmse_avg


# pbounds = {'log_alpha': (-8, 8)}
pbounds = {'alpha': (0, 4)}
bounds_transformer = SequentialDomainReductionTransformer()
optimizer = BayesianOptimization(
    f=loss_fn,
    pbounds=pbounds,
    # bounds_transformer = bounds_transformer,
    random_state=0
    )

# load_logs(optimizer, logs=["bayes-opt-log-2.json"]);

# logger = JSONLogger(path="bayes-opt-lasso.json")
# optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

optimizer.maximize(init_points=5, n_iter=45,
                   acq='ei', xi=1e-2,
                #    acq='ucb', kappa=4,
                   )

print(optimizer.max)

|   iter    |  target   |   alpha   |
-------------------------------------
|  1        | -2.876    |  2.195    |
|  2        | -2.883    |  2.861    |
|  3        | -2.878    |  2.411    |
|  4        | -2.876    |  2.18     |
|  5        | -2.871    |  1.695    |
|  6        | -2.862    |  0.4356   |
|  7        | -2.862    |  0.000665 |
|  8        | -2.895    |  3.999    |
|  9        | -2.865    |  0.9651   |
|  10       | -2.884    |  2.958    |
|  11       | -2.866    |  1.149    |
|  12       | -2.894    |  3.901    |
|  13       | -2.887    |  3.206    |
|  14       | -2.876    |  2.177    |
|  15       | -2.889    |  3.409    |
|  16       | -2.862    |  0.3493   |
|  17       | -2.886    |  3.152    |
|  18       | -2.883    |  2.817    |
|  19       | -2.866    |  1.181    |
|  20       | -2.875    |  2.133    |
|  21       | -2.885    |  3.068    |
|  22       | -2.877    |  2.287    |
|  23       | -2.888    |  3.361    |
|  24       | -2.862    |  0.3229   |
|  25       

In [ ]:
np.exp(optimizer.max['params']['log_alpha'])

0.00033546262790251185

In [ ]:
loss_fn(**{'alpha': 0})

-2.8622375238163715

In [ ]:
this_data = fold_data[0]

model = linear_model.LinearRegression()

this_X = this_data['train']['X']
this_y = this_data['train']['y']

train_X = np.vstack([this_X, np.fliplr(this_X)])
train_y = np.vstack([this_y, this_y])

%timeit model.fit(train_X, train_y)

test_X = this_data['test']['X']
test_y = this_data['test']['y']

%timeit model.predict(test_X)

100 loops, best of 5: 3.42 ms per loop
The slowest run took 4.00 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 53.6 µs per loop


## linear regression for k-mers over folds

In [ ]:
all_rmse = []

for i, this_data in enumerate(token_fold_data):

    model = linear_model.LinearRegression()

    this_X = this_data['train']['X']
    this_y = this_data['train']['y']

    train_X = np.vstack([this_X, np.fliplr(this_X)])
    train_y = np.vstack([this_y, this_y])

    model.fit(train_X, train_y)

    test_X = this_data['test']['X']
    test_y = this_data['test']['y']

    # forward sequences
    pred_z_fwd = model.predict(test_X)

    # reverse sequences
    pred_z_rev = model.predict(np.fliplr(test_X))

    # averarage the two predictions
    rmse_avg = np.sqrt(np.mean((0.5*(pred_z_fwd + pred_z_rev) - test_y)**2))

    all_rmse.append(rmse_avg)

print(np.mean(all_rmse), np.std(all_rmse))

3.059670594243504 0.14250770876776345


Linear: 3.0157332226636155 0.22095644516346405

Ridge: 2.780543994395317 0.1583177031987842


## k-neighbors

In [ ]:
from sklearn import neighbors


prev_results = {}
this_data = token_fold_data[0]

def loss_fn(**params):
    params = {k: int(np.round(v)) for k, v in params.items()}
    if params['weights'] > 0.5:
        params['weights'] = 'uniform'
    else:
        params['weights'] = 'distance'

    tuple_key = tuple([params[k] for k in sorted(params.keys())])
    if tuple_key in prev_results:
        return prev_results[tuple_key]

    model = neighbors.KNeighborsRegressor(**params)

    this_X = this_data['train']['X']
    this_y = this_data['train']['y']

    train_X = np.vstack([this_X, np.fliplr(this_X)])
    train_y = np.vstack([this_y, this_y])

    model.fit(train_X, train_y)

    test_X = this_data['test']['X']
    test_y = this_data['test']['y']

    # forward sequences
    pred_z_fwd = model.predict(test_X)

    # reverse sequences
    pred_z_rev = model.predict(np.fliplr(test_X))

    # averarage the two predictions
    rmse_avg = np.sqrt(np.mean((0.5*(pred_z_fwd + pred_z_rev) - test_y)**2))

    # save the results
    prev_results[tuple_key] = -rmse_avg

    return -rmse_avg


pbounds = {'n_neighbors': (1, 128), 'weights': (0, 1)}
# pbounds = {'alpha': (0, 4)}
bounds_transformer = SequentialDomainReductionTransformer()
optimizer = BayesianOptimization(
    f=loss_fn,
    pbounds=pbounds,
    # bounds_transformer = bounds_transformer,
    random_state=0
    )

# load_logs(optimizer, logs=["bayes-opt-log-2.json"]);

# logger = JSONLogger(path="bayes-opt-lasso.json")
# optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

optimizer.maximize(init_points=5, n_iter=45,
                   acq='ei', xi=1e-2,
                #    acq='ucb', kappa=4,
                   )

params = optimizer.max['params']
if params['weights'] > 0.5:
    params['weights'] = 'uniform'
else:
    params['weights'] = 'distance'
print(optimizer.max['target'], params)

|   iter    |  target   | n_neig... |  weights  |
-------------------------------------------------
|  1        | -3.733    |  70.7     |  0.7152   |
|  2        | -3.731    |  77.55    |  0.5449   |
|  3        | -3.71     |  54.8     |  0.6459   |
|  4        | -3.715    |  56.57    |  0.8918   |
|  5        | -3.758    |  123.4    |  0.3834   |
|  6        | -3.651    |  44.58    |  0.008413 |
|  7        | -3.635    |  38.05    |  0.0      |
|  8        | -3.659    |  22.23    |  0.9654   |
|  9        | -5.068    |  1.007    |  0.2338   |
|  10       | -3.772    |  101.7    |  1.0      |
|  11       | -3.691    |  28.5     |  1.0      |
|  12       | -3.728    |  89.86    |  0.0      |
|  13       | -3.753    |  113.5    |  0.04018  |
|  14       | -3.691    |  40.51    |  1.0      |
|  15       | -3.616    |  24.37    |  0.01294  |
|  16       | -3.8      |  128.0    |  1.0      |
|  17       | -3.645    |  34.04    |  0.0      |
|  18       | -3.687    |  63.5     |  0.0      |


In [ ]:
loss_fn(**{'weights': 1})

-3.592616952617676

In [ ]:
from sklearn import neighbors

rmse = []
for this_data in token_fold_data:
    model = neighbors.KNeighborsRegressor(n_neighbors=15, weights='distance')

    this_X = this_data['train']['X']
    this_y = this_data['train']['y']

    train_X = np.vstack([this_X, np.fliplr(this_X)])
    train_y = np.vstack([this_y, this_y])

    model.fit(train_X, train_y)

    test_X = this_data['test']['X']
    test_y = this_data['test']['y']

    # forward sequences
    pred_z_fwd = model.predict(test_X)

    # reverse sequences
    pred_z_rev = model.predict(np.fliplr(test_X))

    # averarage the two predictions
    rmse_avg = np.sqrt(np.mean((0.5*(pred_z_fwd + pred_z_rev) - test_y)**2))

    rmse.append(rmse_avg)

print(np.mean(rmse), np.std(rmse))

3.68340842867823 0.12327479452220283


In [ ]:
print(np.mean(rmse), np.std(rmse))

3.662855428853419 0.13203350578962403


## random forest

In [ ]:
from sklearn import ensemble


prev_results = {}
this_data = token_fold_data[0]

named_params = {'bootstrap': [False, True],
                'max_features': ['auto', 'sqrt']}

def loss_fn(**params):
    params = {k: int(np.round(v)) for k, v in params.items()}
    for key in named_params.keys():
        params[key] = named_params[key][params[key]]

    tuple_key = tuple([params[k] for k in sorted(params.keys())])
    if tuple_key in prev_results:
        return prev_results[tuple_key]

    model = ensemble.RandomForestRegressor(**params)

    this_X = this_data['train']['X']
    this_y = this_data['train']['y']

    train_X = np.vstack([this_X, np.fliplr(this_X)])
    train_y = np.vstack([this_y, this_y])

    model.fit(train_X, train_y)

    test_X = this_data['test']['X']
    test_y = this_data['test']['y']

    # forward sequences
    pred_z_fwd = model.predict(test_X)

    # reverse sequences
    pred_z_rev = model.predict(np.fliplr(test_X))

    # averarage the two predictions
    rmse_avg = np.sqrt(np.mean((0.5*(pred_z_fwd + pred_z_rev) - test_y)**2))

    # save the results
    prev_results[tuple_key] = -rmse_avg

    return -rmse_avg


pbounds = {'max_depth': (2, 32), 'n_estimators': (4, 128),
           'min_samples_leaf': (2, 16), 'min_samples_split': (2, 16),
           'bootstrap': (0, 1), 'max_features': (0, 1)}
bounds_transformer = SequentialDomainReductionTransformer()
optimizer = BayesianOptimization(
    f=loss_fn,
    pbounds=pbounds,
    bounds_transformer = bounds_transformer,
    random_state=0
    )

optimizer.maximize(init_points=5, n_iter=45,
                   acq='ei', xi=1e-2,
                   )

print(optimizer.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        | -3.012    |  18.46    |  12.01    |  10.44    |  71.57    |
|  2        | -3.005    |  14.71    |  11.04    |  8.126    |  114.6    |
|  3        | -2.924    |  30.91    |  7.368    |  13.08    |  69.58    |
|  4        | -3.11     |  19.04    |  14.96    |  2.995    |  14.8     |
|  5        | -4.005    |  2.607    |  13.66    |  12.89    |  111.9    |
|  6        | -2.982    |  29.26    |  9.535    |  14.06    |  70.22    |
|  7        | -2.855    |  25.81    |  4.156    |  10.46    |  66.38    |
|  8        | -2.897    |  30.16    |  5.054    |  8.227    |  59.35    |
|  9        | -2.862    |  21.88    |  2.0      |  12.47    |  57.1     |
|  10       | -2.849    |  29.34    |  2.0      |  8.651    |  80.29    |
|  11       | -2.852    |  32.0     |  2.0      |  7.114    |  94.64    |
|  12       | -2.879    |  32.0     | 

In [ ]:
loss_fn(**{})

-2.8649303153066294

# timings

In [ ]:
from sklearn import neural_network

this_data = token_fold_data[0]

# model = ensemble.RandomForestRegressor()
# model = neighbors.KNeighborsRegressor(weights='distance')
model = neural_network.MLPRegressor(max_iter=2000, random_state=0)

this_X = this_data['train']['X']
this_y = this_data['train']['y']

train_X = np.vstack([this_X, np.fliplr(this_X)])
train_y = np.vstack([this_y, this_y])

%timeit model.fit(train_X, train_y)

test_X = this_data['test']['X']
test_y = this_data['test']['y']

%timeit model.predict(test_X)

1 loop, best of 5: 3min 19s per loop
100 loops, best of 5: 2.42 ms per loop
